In [2]:
pip uninstall -y foresight 

Found existing installation: foresight 0.3.2
Uninstalling foresight-0.3.2:
  Successfully uninstalled foresight-0.3.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
# !pip install -U git+https://github.com/SAGE-3/next.git@dev#subdirectory=foresight

In [3]:
# # from foresight.config import config as conf, prod_type
# import json
# import copy
# import httpx
# from uuid import UUID
# from langchain.prompts import (
#     PromptTemplate,
#     ChatPromptTemplate,
#     HumanMessagePromptTemplate,
# )
# from langchain.output_parsers import PydanticOutputParser
# from langchain.prompts.few_shot import FewShotPromptTemplate
# from langchain_openai import ChatOpenAI
# from typing import TypedDict, Annotated, Sequence
# import operator
# from langchain_core.messages import BaseMessage

from foresight.Sage3Sugar.pysage3 import PySage3
import getpass
import os

from pydantic import BaseModel, Field, UUID4
from typing import Optional, Union, Literal, List


from langchain.pydantic_v1 import BaseModel, Field, UUID4
# from uuid import uuid4

# httpx_client = httpx.Client(timeout=None)


In [4]:
keys_to_remove = [
    'touched', 'executeInfo', 'path', 'raised', 
    'title', 'roomId', 'boardId', 'lock', 'dragging', 'pinned', 'rotation'
]

# Classes

In [5]:

class Size(BaseModel):
    """
    The dimensions of the app
    """
    width: int = Field(description="The width of the app")
    height: int = Field(description="The height of the app")
    depth: int = Field(description="The depth of the app")

class Position(BaseModel):
    """
    The position of the app on the board
    """
    x: int = Field(description="The x position of the app")
    y: int = Field(description="The y position of the app")
    z: int = Field(description="The z position of the app")

class Data(BaseModel):
    position: Position = Field(description="The position of the stickie note on the board")
    size: Size = Field(description="The dimensions of the stickie note on the board")
    type: Literal['Stickie', 'PDFViewer', 'Counter', 'Slider', 'VegaViewer'] = Field(description="The type of the app represented. For example, Stickie for stickie note, PDFViewer, etc.")

class StickieState(BaseModel):
    text: str = Field(description="The text to display on the stickie note")
    color: str = Field(description="The background color of the stickie note, use yellow if a color is not provided")
    fontSize: int = Field(description="The font size to use for the text")

class CounterState(BaseModel):
    count: int = Field(description="The count to display on the counter")

class PDFViewerState(BaseModel):
    currentPage: int = Field(description="The page number currently showing")
    numPages: int = Field(description="The total number of pages in the pdf document")
    displayPages: int = Field(description="The number of pages to display at a time")
    analyzed: str = Field(description="Whether the pdf was converted to text")
    client: str = Field(description="The client used. This is currently set to empty")

class SmartBit(BaseModel):
    app_id: str = Field(description="The UUID4 of this asset")
    data: Data = Field(description="Generic app data like position, width and height")
    state: Union[StickieState, CounterState, PDFViewerState] = Field(description="Data specific to the app type like color of a stickie or page currently being viewed for a PDF viewer")
    tags: List[str] = Field(default_factory=list, description= "List of tag assigned to this app")

# Update forward references to resolve ForwardRef issues
SmartBit.update_forward_refs()





# Helper Functions & Objects

In [6]:
import copy

def remove_keys_from_dict(app_dict, keys_to_remove):
    def _remove_keys_from_dict(d, keys_to_remove):
        if isinstance(d, dict):
            for key in list(d.keys()):  # Use list() to avoid 'dictionary changed size during iteration' error
                if key in keys_to_remove:
                    del d[key]
                else:
                    _remove_keys_from_dict(d[key], keys_to_remove)
        elif isinstance(d, list):
            for item in d:
                _remove_keys_from_dict(item, keys_to_remove)

    # Create a deep copy of the input dictionary
    app_dict_copy = copy.deepcopy(app_dict)
    # Remove keys from the copied dictionary
    _remove_keys_from_dict(app_dict_copy, keys_to_remove)
    return app_dict_copy


In [7]:
import os
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [7]:

# llm_4o = ChatOpenAI(temperature=0.1, model="gpt-4o")
# llm_4o.invoke("hello")

In [8]:
conf = {'local': {'seer_server': 'http://127.0.0.1:5002',
  'jupyter_server': 'http://localhost:8888',
  'redis_server': 'localhost',
  'web_server': 'http://localhost:3333',
  'ws_server': 'ws://localhost:3333'}}
prod_type= 'local'
room_id = '09073d6d-6d79-4c98-bbea-4dd82106c098'
board_id = '68993aeb-0ee6-4f95-b461-0f918fc7d969'
app_id = '4a29b8bf-9e72-4366-8946-d808e2e3d465'

ps3 = PySage3(conf, prod_type)

Configuring ps3 client ... 
Completed configuring Sage3 Client


In [12]:
# # text-containing fields
# {
#     "stickie": "state.text"    
# }

# # assetid-containing field
# {
#     "pdf": "state.assetid"    
# }

In [ ]:
# def get_alltags():
#     """Return all the tags associated with a room"""
#     res = httpx_client.get(
#         conf[prod_type]["web_server"] + "/api/insight/",
#         headers={"Authorization": f"Bearer {os.getenv('TOKEN')}"},
#     )
#     if res.is_success:
#         d = res.json()['data']
#     d = [x['data'] for x in d if x['data']['labels']]    
#     result = {}
#     for item in d:
#         app_id = item.pop('app_id')
#         result[app_id] = item
#     return result


In [ ]:
# def get_all_assets(asset_id=None):
#     """Return all assets in a room or only that with asset_id if not None"""
#     res = httpx_client.get(
#         conf[prod_type]["web_server"] + "/api/assets/",
#         headers={"Authorization": f"Bearer {os.getenv('TOKEN')}"},
#     )
#     if res.is_success:
#         d = res.json()['data']
#         d = {x["_id"]:x for x in d}
#         if asset_id is not None and asset_id in d:
#             d = {asset_id: d[asset_id]}
#         return d
#     else:
#         return None

In [ ]:
# len(get_all_assets())
# len(get_all_assets('ba45e317-e3f7-49c6-ad80-55893500ba0d'))

In [ ]:
# class AgentState(TypedDict):
#     _input: str
#     messages: Annotated[Sequence[BaseMessage], operator.add]

In [ ]:
# def get_pdf_text(asset_url):
#     """Get one page for now"""
#     file_name = asset_url.split("/")[-1].split(".")[0]+"-text.json"
#     file_url = conf[prod_type]["web_server"] + '/api/assets/static/' + file_name
#     res = httpx_client.get(
#         file_url,
#         headers={"Authorization": f"Bearer {os.getenv('TOKEN')}"},
#     )
#     if res.is_success:
#         return res.json()['pages'][0]
#     return None

def get_apps_text(apps):
    text = ""
    for app in apps.values():
        app_type = app.get('data', {}).get('type', None)
        
        if app_type is None:            
            return "AppType Not Valid"
        elif app_type == 'Stickie':
            text += app.get('state', {}).get('text', "")+"\n"
        elif app_type == 'PDFViewer':
            asset_id = app.get('state', {}).get('assetid')
            asset_info = get_all_assets(asset_id)
            asset_url = asset_info[asset_id]['data']["path"]
            text += get_pdf_text(asset_url)

        else:
            return "Cannot yet summarize {app_type}"
    return text



# Tools Section

In [26]:
example_1 = """[{{'app_id': 'ba88e0df-d885-4ce3-ad74-006fbba69b43',
  'data': {{'position': {{'x': 1502426, 'y': 1500697, 'z': 0}},
   'size': {{'width': 400, 'height': 517, 'depth': 0}},
   'rotation': {{'x': 0, 'y': 0, 'z': 0}},
   'type': 'PDFViewer'}},
  'state': {{'assetid': '4d2acd83-8fa2-4c7d-8e91-8e2e8cf45179',
   'currentPage': 0,
   'numPages': 8,
   'displayPages': 1,
   'analyzed': '',
   'client': ''}},
 {{'app_id': '0fbbc02b-6f62-4bbe-8bfb-b6698b7da401',
  'data': {{'position': {{'x': 1502778, 'y': 1500423, 'z': 0}},
   'size': {{'width': 400, 'height': 420, 'depth': 0}},
   'rotation': {{'x': 0, 'y': 0, 'z': 0}},
   'type': 'Stickie'}},
  'state': {{'text': 'Philosophy', 'fontSize': 36, 'color': 'yellow'}},
 {{'app_id': '9ae9a2eb-d901-44c8-83e3-5b447043fd67',
  'data': {{'position': {{'x': 1503749, 'y': 1500342, 'z': 0}},
   'size': {{'width': 400, 'height': 420, 'depth': 0}},
   'rotation': {{'x': 0, 'y': 0, 'z': 0}},
   'type': 'Stickie'}},
  'state': {{'text': 'Tech and AI', 'fontSize': 36, 'color': 'red'}}
]
"""



In [27]:
example_1

"[{{'app_id': 'ba88e0df-d885-4ce3-ad74-006fbba69b43',\n  'data': {{'position': {{'x': 1502426, 'y': 1500697, 'z': 0}},\n   'size': {{'width': 400, 'height': 517, 'depth': 0}},\n   'rotation': {{'x': 0, 'y': 0, 'z': 0}},\n   'type': 'PDFViewer'}},\n  'state': {{'assetid': '4d2acd83-8fa2-4c7d-8e91-8e2e8cf45179',\n   'currentPage': 0,\n   'numPages': 8,\n   'displayPages': 1,\n   'analyzed': '',\n   'client': ''}},\n {{'app_id': '0fbbc02b-6f62-4bbe-8bfb-b6698b7da401',\n  'data': {{'position': {{'x': 1502778, 'y': 1500423, 'z': 0}},\n   'size': {{'width': 400, 'height': 420, 'depth': 0}},\n   'rotation': {{'x': 0, 'y': 0, 'z': 0}},\n   'type': 'Stickie'}},\n  'state': {{'text': 'Philosophy', 'fontSize': 36, 'color': 'yellow'}},\n {{'app_id': '9ae9a2eb-d901-44c8-83e3-5b447043fd67',\n  'data': {{'position': {{'x': 1503749, 'y': 1500342, 'z': 0}},\n   'size': {{'width': 400, 'height': 420, 'depth': 0}},\n   'rotation': {{'x': 0, 'y': 0, 'z': 0}},\n   'type': 'Stickie'}},\n  'state': {{'text':

In [ ]:

# select_apps_fewshot_prompt = 'my example'

In [ ]:
# # from typing import List, Union
# # from uuid import UUID, uuid4
# # from langchain.prompts import PromptTemplate, FewShotPromptTemplate
# # from langchain.pydantic_v1 import BaseModel, Field
# # from langchain.output_parsers import PydanticOutputParser

# # # Example data


# # # Prefix for the few-shot prompt
# # select_apps_prefix = """
# # Given the list of items below, identify the app_ids (UUID4) of the items that match the user's query defined below. 
# # If none of the items match the query, return an empty list.


# # {format_instructions}

# # Example:
# # """

# # # Template for examples
# # select_apps_examples_template = """
# # originalItems: {originalItems}

# # query: {query}

# # ```
# # json
# # {{{{ 
# #   "uuids": {matchedItems}
# # }}}}
# # ```
# # """

# # # Example data for the few-shot prompt
# # select_apps_examples_data = [
# #     {
# #         "originalItems": example_1,
# #         "query": "Which stickies are red",
# #         "matchedItems": "['9ae9a2eb-d901-44c8-83e3-5b447043fd67']"
# #     }
# # ]

# # Example prompt template
# select_apps_examples_prompt = PromptTemplate(
#     input_variables=["originalItems", "query", "matchedItems"],
#     template=select_apps_examples_template,
# )

# # Define output model
# class SelectAppsOutput(BaseModel):
#     uuids: List[UUID] = Field(description="A valid Python list of UUIDs selected")

# # Create an output parser
# select_apps_output_parser = PydanticOutputParser(pydantic_object=SelectAppsOutput)

# # Few-shot prompt template
# select_apps_fewshot_prompt = FewShotPromptTemplate(
#     input_variables=["originalItems", "query"],
#     partial_variables={
#         "format_instructions": select_apps_output_parser.get_format_instructions(),
#         "open_curly": "{{",
#         "close_curly": "}}"
#     },
#     examples=select_apps_examples_data,
#     example_prompt=select_apps_examples_prompt,
#     prefix=select_apps_prefix,
#     suffix="originalItems: {originalItems}\n\nquery: {query}\n\n```json",
#     example_separator="\n"
# )

# # Print the formatted few-shot prompt
# formatted_prompt = select_apps_fewshot_prompt.format(
#     originalItems=example_1,
#     query="Which stickies talk about science"
# )
# print(formatted_prompt)


In [ ]:
# class ReframeSelectOutput(BaseModel):
#     updated_query: str = Field(description="The new reframed query str describing the items referenced in this query")

# reframeSelectOutput = PydanticOutputParser(pydantic_object=ReframeSelectOutput)
# reframe_select_prompt = """You are a meta-agent that needs to instruct a specialized agent to select the items to which a 
# query applies in a new updated_query. The new_updated query serves to identify the items which are first loaded.
# A second agent then completes the desired action.

# {format_instructions}

# Example: 
# query: Can you summarize all the red stickies.
# updated_query: Select the red stickies.

# query: can you delete all the pdf files.
# updated_query: select the pdf files.

# query: Can you change the color of purple stickies to reda
# updated_query: select purple stickies

# query: {query}
# """ 


# reframe_select_prompt_template = PromptTemplate(
#     template= reframe_select_prompt,
#     input_variables=["query"],
#     partial_variables={
#         "format_instructions": reframeSelectOutput.get_format_instructions(),
#     },
#     suffix="```json",
# )


# select_items_chain = reframe_select_prompt_template | llm_4o | reframeSelectOutput
# x = select_items_chain.invoke({"query": "Can you move the red stickies to the left of the screen?"})


# Temp Tests

In [ ]:
all_tags = get_alltags()
all_tags


In [ ]:
def format_smartbits_with_tags():
    all_tags = get_alltags()
    all_apps = [remove_keys_from_dict(x[1].dict(), keys_to_remove) for x in list(cb.smartbits)]
    for app in all_apps:
        if app['app_id'] in all_tags:
            app['tags'] = all_tags[app['app_id']]['labels']
        else:
            app['tags'] = []
            
    all_apps  = {x['app_id']: x for x in all_apps}    
    return all_apps

In [ ]:
format_smartbits_with_tags()

In [ ]:
formatted_prompt = select_apps_fewshot_prompt.format(
    originalItems=all_apps,
    query="The apps representing stickies"
)
print(formatted_prompt)

In [ ]:
all_apps 

In [ ]:
q = "Select all stickies"
filter_app_chain = select_apps_fewshot_prompt | llm_4o | select_apps_output_parser
selected_uuids = filter_app_chain.invoke({"originalItems": all_apps, "query": q})
selected_uuids

In [ ]:
selected_uuids.uuids

In [ ]:
selected_uuids.uuids[0] == UUID('0fbbc02b-6f62-4bbe-8bfb-b6698b7da401')

In [ ]:
selected_apps = {k:v for k,v in all_apps.items() if UUID(k) in  selected_uuids.uuids}
selected_apps

In [ ]:

x = get_all_assets(asset_id)
get_pdf_text(x[asset_id]['data']["path"])[0:40]

In [ ]:
print(get_apps_text(selected_apps))

#  Ranom Data Used For tests

In [ ]:
asset_id = '4d2acd83-8fa2-4c7d-8e91-8e2e8cf45179'

In [ ]:
x = get_all_assets(asset_id)
x[asset_id]['data']["path"].split("/")[-1].split(".")[0]+"-text.json"

# Complete Example

In [ ]:
all_tags = get_alltags()
all_tags

In [ ]:
cb = ps3.rooms['09073d6d-6d79-4c98-bbea-4dd82106c098'].boards['68993aeb-0ee6-4f95-b461-0f918fc7d969']

all_tags = get_alltags()
all_apps = [remove_keys_from_dict(x[1].dict(), keys_to_remove) for x in list(cb.smartbits)]

for app in all_apps:
    if app['app_id'] in all_tags:
        app['tags'] = all_tags[app['app_id']]['labels']
    else:
        app['tags'] = []
        
all_apps  = {x['app_id']: x for x in all_apps}    
all_apps

In [ ]:
q = "Select the pdf"
filter_app_chain = select_apps_fewshot_prompt | llm_4o | select_apps_output_parser
selected_uuids = filter_app_chain.invoke({"originalItems": all_apps, "query": q})
selected_uuids

In [ ]:
selected_apps = {k:v for k,v in all_apps.items() if UUID(k) in  selected_uuids.uuids}
selected_apps

In [ ]:
# ps3.create_app()

In [ ]:
cell = list(ps3.get_smartbits(room_id, board_id))[1][1]
pad = 20
psize = [cell.data.position.x, cell.data.position.y, cell.data.size.width, cell.data.size.height]
right = {'x': psize[0] + psize[2] + pad, 'y': psize[1], 'z': 0}
psize, right


In [ ]:
ps3.create_app('09073d6d-6d79-4c98-bbea-4dd82106c098', 
               '68993aeb-0ee6-4f95-b461-0f918fc7d969', 
               'Stickie', 
                {'text': 'hello', 'color': 'red', 'fontSize':36}, 
                {
                    'position': {"x":1504150,"y":1501300,"z":0}
                }
)

In [ ]:
# (room_id, board_id, app_type, state, app=None)

In [ ]:
SmartBit(

In [22]:
from uuid import uuid4

# Example of creating a SmartBit of type Stickie
stickie_smartbit = SmartBit(
    app_id=str(uuid4()),
    data=Data(
        position=Position(x=1504200, y=1501350, z=0),
        size=Size(width=400, height=420, depth=0),
        type="Stickie"
    ),
    state=StickieState(
        text="This is a sample stickie note",
        color="yellow",
        fontSize=24
    ),
    tags=["example", "stickie"]
)

print(stickie_smartbit.json(indent=4))

{
    "app_id": "9a06038d-5503-46a0-900a-01965afe922a",
    "data": {
        "position": {
            "x": 1504200,
            "y": 1501350,
            "z": 0
        },
        "size": {
            "width": 400,
            "height": 420,
            "depth": 0
        },
        "type": "Stickie"
    },
    "state": {
        "text": "This is a sample stickie note",
        "color": "yellow",
        "fontSize": 24
    },
    "tags": [
        "example",
        "stickie"
    ]
}


In [ ]:
stickie_smartbit.dict()

In [ ]:
x = stickie_smartbit.dict()


# x['_id'] = x['app_id']
del x['tags']
del x['app_id']
x

In [ ]:
x = x['data']['type']
x

In [ ]:
x = {
 'data': {'title': 'Mahdi Bel-caid',
  'position': {'x': 1503627, 'y': 1500957, 'z': 0},
  'size': {'width': 400, 'height': 420, 'depth': 0},
  'rotation': {'x': 0, 'y': 0, 'z': 0},
  'type': 'Stickie',
  'state': {'text': 'My third stickie is about politics',
   'fontSize': 36,
   'color': 'blue',
   'lock': False,
   'executeInfo': {'executeFunc': '', 'params': {}}},
  'raised': True,
  'dragging': False,
  'pinned': False}}

x, x['data']['type']


In [ ]:
data = {key: x['data'][key] for key in ['position', 'size', 'rotation']}

ps3.create_app(room_id, board_id, x['data']['type'], x['data']['state'], data)

In [ ]:
smartbit_output_parser.get_format_instructions()

In [ ]:
all_apps = format_smartbits_with_tags()
all_apps

In [ ]:
smartbit_output_parser = PydanticOutputParser(pydantic_object=SmartBit)

wall_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert operator that know how to create, delete and query apps on the large display operating system "
            "referred to as the Wall or SAGE or SAGE3. Given the user query, you need to identify all  "
            "the necessary fields and format the call to provide the appropriate information. Also, if not position is provided, you need to "
            "find a good position to create the new app without overlapping existing items but without leaving a lot of unused space."
            "\n"
             "{format_instructions}" 
            "\n"
            "The list of other application currently being displayed is"
            "{all_apps}",
        
        ),
        ("placeholder", "{messages}"),
    ]
).partial(format_instructions= smartbit_output_parser.get_format_instructions())

print(wall_assistant_prompt.invoke({"messages": ["HI", ("system", "There")], "all_apps": all_apps}))


In [ ]:
sb.dict()

In [4]:
from langchain_core.tools import tool

# def get_request_info():
#     return 
#     {
#         "room_id": '09073d6d-6d79-4c98-bbea-4dd82106c098',
#         "board_id": '68993aeb-0ee6-4f95-b461-0f918fc7d969',
#         "user_name": "mahdi"
#     }

config = {"configurable": {
        "room_id": room_id,
        "wall_id": wall_id
    }
}

@tool
def execute_create_app(sb: SmartBit) -> Literal["Success", "Failure"]:
    """ Takes a input consisting in a list of valid smartbit JSON object and creates it on the wall"""
    print("I am in the create_app function")
    print("The value of sb is {sb}")

    config = ensure_config()  # Fetch from the context
    configuration = config.get("configurable", {})
    room_id = configuration.get(room_id, None)
    wall_id = configuration.get(wall_id, None)
    
    if room_id is None or wall_id is None:
        raise Exception('Wall or Room Ids not found in config')

    app_type = sb.data.type
    data= sb.data.dict()
    del data['type']
    
    del sb.data.dict()['type']
    ps3.create_app(room_id, wall_id,  app_type, sb.state.dict(), data)

    
 #    {'name': 'create_app',
 # 'args': {'sb': {'app_id': 'a7f7bcd6-2065-4eeb-823f-d6e2d837743c',
 #   'data': {'position': {'x': 1504550, 'y': 1501300, 'z': 0},
 #    'size': {'width': 400, 'height': 400, 'depth': 0},
 #    'type': 'Stickie'},
 #   'state': {'text': 'hello', 'color': 'yellow', 'fontSize': 36},
 #   'tags': []}},
 # 'id': 'call_iDCWPqeyRsljq4xQTWusbEaH'}



@tool
def execute_delete_apps(uuids: UUID) -> Literal["Success", "Failure"]:
    """ Takes one of more UUIDs and deletes them form the wall"""
    print("I am in the delete_app function")
    print(f"The value of sb is {sb}")

@tool
def execute_query_apps(uuids: UUID) -> Literal["Success", "Failure"]:
    """ Takes one of more UUIDs and deletes them form the wall"""
    print("I am in the delete_app function")
    print("The value of sb is {sb}")


llm_4o = ChatOpenAI(model="gpt-4o", temperature=1)
wall_assistant_runnable = wall_assistant_prompt | llm_4o.bind_tools([create_app])    

In [ ]:
all_apps

In [ ]:
all_apps = {}

In [ ]:
q = "Please create Stickies with the names of the 8 wealthiest countries in the world based on GDP (Gross Domestic Product). Arrange the stickies spatially (x and y coordiantes) in positions they would appear geographically on a world map, starting from the westernmost country and moving eastward. Don't prompt me for other info. Proceed with subjective decisions as needed without asking me." 
x = wall_assistant_runnable.invoke({"messages":[q], "all_apps": all_apps})

In [ ]:
x

In [ ]:
x.tool_calls

In [ ]:
len(x.tool_calls)

In [ ]:
for tool_call in x.tool_calls:
    sb = SmartBit(**tool_call['args']['sb'])
    locals()['execute_create_app'].invoke({"sb":sb})

In [13]:
from datetime import datetime
from datetime import date, datetime
from typing import Optional
from typing import Callable

from langchain_core.messages import ToolMessage
from typing import Literal

# from langgraph.checkpoint.sqlite import SqliteSaver
# from langgraph.graph import END, StateGraph
# from langgraph.prebuilt import tools_condition
# from langgraph.prebuilt import ToolNode
# from langchain_core.runnables import RunnableLambda
# from langchain_core.messages import ToolMessage

# import pytz
from langchain_core.runnables import ensure_config
import getpass
import os
import shutil
import sqlite3
from typing import Annotated, Literal, Optional
from typing_extensions import TypedDict
# from langgraph.graph.message import AnyMessage, add_messages

# import pandas as pd
import requests
from langchain_core.pydantic_v1 import BaseModel, Field

In [14]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import Runnable, RunnableConfig


llm = ChatOpenAI(model="gpt-4o", temperature=0)


In [15]:
from typing import Literal, Dict
from langchain_core.tools import BaseTool
from typing import Optional, Type
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

WidgetType = Literal['Stickie', 'PDFViewer', 'Counter', 'Slider', 'VegaViewer']
smartbit_dict_example = {
    "app_id": "9a06038d-5503-46a0-900a-01965afe922a",
    "data": {
        "position": {
            "x": 1504200,
            "y": 1501350,
            "z": 0
        },
        "size": {
            "width": 400,
            "height": 420,
            "depth": 0
        },
        "type": "Stickie"
    },
    "state": {
        "text": "This is a sample stickie note",
        "color": "yellow",
        "fontSize": 24
    },
    "tags": [
        "example",
        "stickie"
    ]
}


class CreateAppToolInput(BaseModel):
        app_type: WidgetType =   Field(description="The type of the app to create")
        smart_bit: SmartBit  =   Field(description="The smart bit data used to instantiate the app")
        class Config:
            json_schema_extra = {
            "example": {
                "app_type": "Stickie",
                "data": smartbit_dict_example
            }
        }

        
class CreateAppTool(BaseTool):
    args_schema: Type[BaseModel] = CreateAppToolInput
        
    name: str = "CreateAppTool"
    description: str = "Create an app on the wall given an app type and the data that defines the app"


    def _run(
        self, origin: str, destination: str, departure_date: str, return_date: str, request: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        print("Running Create app")
        return "Done with CreateAppTool"


In [16]:
llm_with_create = llm.bind_tools([CreateAppTool()])
resp = llm_with_create.invoke("create a stickie with text hello world in english french and italian and color red and font size 36")
resp

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_h5VjNNjs7js5L2Lla1keNUWL', 'function': {'arguments': '{"app_type":"Stickie","smart_bit":{"app_id":"1","data":{"position":{"x":100,"y":100,"z":0},"size":{"width":200,"height":200,"depth":1},"type":"Stickie"},"state":{"text":"Hello World\\nBonjour le monde\\nCiao mondo","color":"red","fontSize":36}}}', 'name': 'CreateAppTool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 584, 'total_tokens': 670}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6d501291-882f-47ef-9ba2-80aa355d72c9-0', tool_calls=[{'name': 'CreateAppTool', 'args': {'app_type': 'Stickie', 'smart_bit': {'app_id': '1', 'data': {'position': {'x': 100, 'y': 100, 'z': 0}, 'size': {'width': 200, 'height': 200, 'depth': 1}, 'type': 'Stickie'}, 'state': {'text': 'Hello World\nBonjour le monde\nCiao mondo', 'color': 'red', 'fontSize': 36

In [17]:
resp.tool_calls

[{'name': 'CreateAppTool',
  'args': {'app_type': 'Stickie',
   'smart_bit': {'app_id': '1',
    'data': {'position': {'x': 100, 'y': 100, 'z': 0},
     'size': {'width': 200, 'height': 200, 'depth': 1},
     'type': 'Stickie'},
    'state': {'text': 'Hello World\nBonjour le monde\nCiao mondo',
     'color': 'red',
     'fontSize': 36}}},
  'id': 'call_h5VjNNjs7js5L2Lla1keNUWL'}]

In [18]:
room_id = "d6640933-7b9f-4536-a263-02a514ff092a"
wall_id = "02fc98df-b4f1-470c-8516-7221c3095de7"
config = {"configurable": {
        "room_id": room_id,
        "wall_id": wall_id
    }
}

In [11]:
# ps3.create_app(room_id, wall_id,  app_type, sb.state.dict(), data)

In [19]:
def execute_create_app(sb: SmartBit) -> Literal["Success", "Failure"]:
    """ Takes a input consisting in a list of valid smartbit JSON object and creates it on the wall"""
    print("I am in the create_app function")
    print("The value of sb is {sb}")

#     config = ensure_config()  # Fetch from the context
    global config
    print(config)
    configuration = config.get("configurable", {})
    room_id = configuration.get("room_id", None)
    wall_id = configuration.get("wall_id", None)
    
    if room_id is None or wall_id is None:
        raise Exception('Wall or Room Ids not found in config')

    app_type = sb.data.type
    data= sb.data.dict()
    del data['type']
    
    del sb.data.dict()['type']
    ps3.create_app(room_id, wall_id,  app_type, sb.state.dict(), data)


In [20]:
sb_dict = resp.tool_calls[0]['args']['smart_bit']
sb = SmartBit(**sb_dict)
execute_create_app(sb)

I am in the create_app function
The value of sb is {sb}
{'configurable': {'room_id': 'd6640933-7b9f-4536-a263-02a514ff092a', 'wall_id': '02fc98df-b4f1-470c-8516-7221c3095de7'}}


In [ ]:
from langchain_core.tools import tool
from langchain_core.tools import BaseTool


# def get_request_info():
#     return 
#     {
#         "room_id": '09073d6d-6d79-4c98-bbea-4dd82106c098',
#         "board_id": '68993aeb-0ee6-4f95-b461-0f918fc7d969',
#         "user_name": "mahdi"
#     }

config = {"configurable": {
        "room_id": room_id,
        "wall_id": wall_id
    }
}
    
    
    
    
    

        
        

def execute_create_app(sb: SmartBit) -> Literal["Success", "Failure"]:
    """ Takes a input consisting in a list of valid smartbit JSON object and creates it on the wall"""
    print("I am in the create_app function")
    print("The value of sb is {sb}")

    config = ensure_config()  # Fetch from the context
    configuration = config.get("configurable", {})
    room_id = configuration.get(room_id, None)
    wall_id = configuration.get(wall_id, None)
    
    if room_id is None or wall_id is None:
        raise Exception('Wall or Room Ids not found in config')

    app_type = sb.data.type
    data= sb.data.dict()
    del data['type']
    
    del sb.data.dict()['type']
    ps3.create_app(room_id, wall_id,  app_type, sb.state.dict(), data)

    
 #    {'name': 'create_app',
 # 'args': {'sb': {'app_id': 'a7f7bcd6-2065-4eeb-823f-d6e2d837743c',
 #   'data': {'position': {'x': 1504550, 'y': 1501300, 'z': 0},
 #    'size': {'width': 400, 'height': 400, 'depth': 0},
 #    'type': 'Stickie'},
 #   'state': {'text': 'hello', 'color': 'yellow', 'fontSize': 36},
 #   'tags': []}},
 # 'id': 'call_iDCWPqeyRsljq4xQTWusbEaH'}



@tool
def execute_delete_apps(uuids: UUID) -> Literal["Success", "Failure"]:
    """ Takes one of more UUIDs and deletes them form the wall"""
    print("I am in the delete_app function")
    print(f"The value of sb is {sb}")

@tool
def execute_query_apps(uuids: UUID) -> Literal["Success", "Failure"]:
    """ Takes one of more UUIDs and deletes them form the wall"""
    print("I am in the delete_app function")
    print("The value of sb is {sb}")


llm_4o = ChatOpenAI(model="gpt-4o", temperature=1)
wall_assistant_runnable = wall_assistant_prompt | llm_4o.bind_tools([create_app])    